This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [41]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [42]:
# First, import the relevant modules
import requests
import math
import collections

In [43]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
params = {'start_date': '2018-10-31', 'end_date': '2018-10-31', 'api_key': API_KEY}
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json', params=params)

In [44]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{u'dataset_data': {u'collapse': None,
  u'column_index': None,
  u'column_names': [u'Date',
   u'Open',
   u'High',
   u'Low',
   u'Close',
   u'Change',
   u'Traded Volume',
   u'Turnover',
   u'Last Price of the Day',
   u'Daily Traded Units',
   u'Daily Turnover'],
  u'data': [[u'2018-10-31',
    None,
    73.25,
    72.2,
    72.45,
    None,
    185365.0,
    13460690.0,
    None,
    None,
    None]],
  u'end_date': u'2018-10-31',
  u'frequency': u'daily',
  u'limit': None,
  u'order': None,
  u'start_date': u'2018-10-31',
  u'transform': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

1: Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [45]:
params_1 = {'start_date': '2017-01-01', 'end_date': '2017-12-31', 'api_key': API_KEY}
r_year = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json', params=params_1)

2: Convert the returned JSON object into a Python dictionary.

In [46]:
fse_dict = dict(r_year.json())

I don't think this is what I wanted. Dictionary omitted to make the file readable, but it looks exactly like r_year.json(), and I can't use column names ('Open', 'Close', etc) to access data.

3: Calculate what the highest and lowest opening prices were for the stock in this period.

In [47]:
fse_data_list = fse_dict['dataset_data']['data']
fse_data_list[0][1]

51.76

In [48]:
opening_prices = []

In [49]:
for daily_list in fse_data_list:
    opening_prices.append(daily_list[1])
    
for i in range(opening_prices.count(None)):
    opening_prices.remove(None)

opening_prices.sort()
print('Lowest Opening Price: ' + str(opening_prices[0]))
print('Highest Opening Price: ' + str(opening_prices[-1]))

Lowest Opening Price: 34.0
Highest Opening Price: 53.11


Below is another way to access a single column (along with the dates), which may be easier? Depends, I guess.

In [50]:
params_open = {'start_date': '2017-01-01', 'end_date': '2017-12-31', 'column_index': 1, 'api_key': API_KEY}
r_opens = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json', params=params_open)

4: What was the largest change in any one day (based on High and Low price)?

In [51]:
high_prices = []
low_prices = []

In [52]:
for daily_list in fse_data_list:
    high_prices.append(daily_list[2])
    low_prices.append(daily_list[3])

In [53]:
price_changes = []
for i in range(len(high_prices)):
    price_changes.append(high_prices[i] - low_prices[i])

price_changes.sort()
price_changes[-1]

2.8100000000000023

5: What was the largest change between any two days (based on Closing Price)?

In [54]:
closing_prices = []
closing_price_changes = []
for daily_list in fse_data_list:
    closing_prices.append(daily_list[4])

In [55]:
for i, price in enumerate(closing_prices):
    if (i != 0):
        closing_price_changes.append(price - closing_prices[i-1])

In [56]:
closing_price_changes.sort()
print('Largest Loss Day-to-Day: ' + str(closing_price_changes[0]))
print('Largest Gain Day-to-Day: ' + str(closing_price_changes[-1]))

Largest Loss Day-to-Day: -1.72
Largest Gain Day-to-Day: 2.56


6: What was the average daily trading volume during this year?

In [57]:
trading_volume = []
for daily_list in fse_data_list:
    trading_volume.append(daily_list[6])

In [58]:
avg_trading_volume = 0
for vol in trading_volume:
    avg_trading_volume += vol
avg_trading_volume /= len(trading_volume)
avg_trading_volume

89124.33725490196

7: What was the median trading volume during this year?

In [59]:
len(trading_volume)/2.0

127.5

In [60]:
trading_volume.sort()
median_trading_volume = 0
if len(trading_volume) % 2 == 0:
    # Even length lists need the avg of the 2 middle values
    median_trading_volume = 0.5 * (
        trading_volume[int(math.ceil(len(trading_volume)/2.0))] +
        trading_volume[int(math.floor(len(trading_volume)/2.0))])
else:
    # Odd length lists need the middle value
    # just use floor
    median_trading_volume = trading_volume[int(math.floor(len(trading_volume)/2.0))]
    
median_trading_volume

76286.0

If I can figure out a better way to use the import the JSON into a dictionary, I may try to do some of the problems again with some cleaner functions.